### 3rd Process:

- From the 80 top players ID's, retrieve all the seasons and teams they participate with API stats=yearByYear
- From API Schedule, get all the games and filter but the teams of the top players to narrow the search field
- Get each play from games, and filter only the games where the top players appears as batters or pitchers.

### Imports

In [1]:
import json
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

In [3]:
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', None)

### Utils

In [4]:


# Function to Process Results from Various MLB Stats API Endpoints
# Provided by the Google Colab from MLB Hackaton

def process_endpoint_url(endpoint_url, pop_key=None):
  """
  Fetches data from a URL, parses JSON, and optionally pops a key.

  Args:
    endpoint_url: The URL to fetch data from.
    pop_key: The key to pop from the JSON data (optional, defaults to None).

  Returns:
    A pandas DataFrame containing the processed data
  """
  json_result = requests.get(endpoint_url).content

  data = json.loads(json_result)

   # if pop_key is provided, pop key and normalize nested fields
  if pop_key:
    df_result = pd.json_normalize(data.pop(pop_key), sep = '_')
  # if pop_key is not provided, normalize entire json
  else:
    df_result = pd.json_normalize(data)

  return df_result

### Get Season and Teams for Top players

In [20]:
top_players_id = pd.read_csv('mlb_top_players_id.csv')
players_id = top_players_id['player_id'].tolist()

In [41]:
def fetch_player_season_team(player_id):
    player_url = f'https://statsapi.mlb.com/api/v1/people/{player_id}/stats?stats=yearByYear'
    player_data = process_endpoint_url(player_url, "stats")

    if not player_data.empty:
        player_data = player_data.loc[0, 'splits']
        player_data_df = (
            pd.DataFrame(player_data)
            .drop_duplicates(subset=['season'], keep='first')
            .assign(
                player_id = player_id,
                team_id = lambda x: x['team'].apply(lambda y: y['id'] if isinstance(y, dict) and 'id' in y else None)
            )
            .filter(['player_id', 'team_id', 'season'])
        )
        return player_data_df
    return pd.DataFrame()

with ThreadPoolExecutor() as executor:
    season_team_results = executor.map(fetch_player_season_team, players_id)

top_players_season_team = pd.concat(season_team_results, ignore_index=True)

In [42]:
top_players_season_team = (
    top_players_season_team
    .merge(
        top_players_id,
        on='player_id'
    )
    .filter([
        'season', 'player_id', 'fullName',
        'team_id', 'top_idx', 'top_source'
    ])
    .astype({'season': 'Int32'})
)

In [ ]:
top_players_season_team.to_csv('mlb_top_players_season_team.csv', index=False)

### Obtener gamePks posibles en season y con el team_id en away o home

In [45]:
top_players_season_team = pd.read_csv('mlb_top_players_season_team.csv')

In [46]:
season_team_id = (
    top_players_season_team
    .drop_duplicates(['season', 'team_id'])
    .groupby('season')['team_id']
    .apply(list)
    .reset_index()
    .sort_values(by=['season'])
)

In [49]:
def fetch_games(season_team_serie: pd.Series) -> pd.DataFrame:

    season = season_team_serie[1]['season']
    team_id = season_team_serie[1]['team_id']

    schedule_endpoint_url = f'https://statsapi.mlb.com/api/v1/schedule?sportId=1&season={season}'
    schedule_dates = process_endpoint_url(schedule_endpoint_url, "dates")

    if not schedule_dates.empty:
        games = (
            pd.json_normalize(
                schedule_dates
                .explode('games')
                .reset_index(drop=True)
                .loc[:,'games']
            )
            .pipe(
                lambda df: df[df['gameType'] == 'R']
            )
            .pipe(lambda df: df[
                df['teams.away.team.id'].isin(team_id) |
                df['teams.home.team.id'].isin(team_id)
            ])
            .filter([
                'gamePk', 'gameGuid', 'season',
                'teams.away.team.id', 'teams.home.team.id'
            ])
        )
        return games
    return pd.DataFrame()

with ThreadPoolExecutor() as executor:
    games_results = executor.map(fetch_games, season_team_id.iterrows())

top_players_games = pd.concat(games_results, ignore_index=True)
top_players_games['season'] = top_players_games['season'].astype('Int32')

In [ ]:
top_players_games.to_csv('mlb_top_players_games.csv', index=False)

### Get plays with participation of top players

In [55]:
top_players_season_team = pd.read_csv('mlb_top_players_season_team.csv')
top_players_games = pd.read_csv('mlb_top_players_games.csv')

In [64]:
batting_season_team = (
    top_players_season_team
    .loc[top_players_season_team['top_source'].isin(['b2024', 'bhist']), ['season', 'player_id']]
    .drop_duplicates()
)

pitching_season_team = (
    top_players_season_team
    .loc[top_players_season_team['top_source'].isin(['p2024', 'phist']), ['season', 'player_id']]
    .drop_duplicates()
)

games_seasons = top_players_games['season'].unique().tolist()

In [68]:
plays_results = []

def extract_gamedata(game_content):

    gamedata_json = game_content.get('gameData', {})

    return {
        'day_night': gamedata_json.get('datetime', {}).get('dayNight', None),
        'venue_capacity': gamedata_json.get('venue', {}).get('fieldInfo', {}).get('capacity', None),
        'venue_turf': gamedata_json.get('venue', {}).get('fieldInfo', {}).get('turfType', None),
        'venue_roof': gamedata_json.get('venue', {}).get('fieldInfo', {}).get('roofType', None),
        'weather_condition': gamedata_json.get('weather', {}).get('condition', None),
        'weather_temp': gamedata_json.get('weather', {}).get('temp', None),
        'attendance': gamedata_json.get('gameInfo', {}).get('attendance', None)
    }

def process_game(game, season, batter_list, pitcher_list):
    single_game_feed_url = f'https://statsapi.mlb.com/api/v1.1/game/{game}/feed/live'
    game_content = json.loads(requests.get(single_game_feed_url).content)
    gamedata_dict = extract_gamedata(game_content)
    livedata_json = game_content.get('liveData', {})
    all_plays = livedata_json.get('plays', {}).get('allPlays', [])

    game_data_list = []

    for play in all_plays:
        batter_id = play.get('matchup', {}).get('batter', {}).get('id', None)
        pitcher_id = play.get('matchup', {}).get('pitcher', {}).get('id', None)
        if any([batter_id in batter_list, pitcher_id in pitcher_list]):
            play_result = play.get('result', {})
            play_about = play.get('about', {})
            play_matchup = play.get('matchup', {})
            play_events = play.get('playEvents', {})
            play_data = pd.DataFrame({
                'season': [season],
                'game_pk': [game],
                **gamedata_dict,
                'batter_id': [batter_id],
                'pitcher_id': [pitcher_id],
                'event_type': [play_result.get('eventType', None)],
                'start_time': [play_about.get('startTime', None)],
                'end_time': [play_about.get('endTime', None)],
                'vs_RHB': [play_matchup.get('batSide', {}).get('code', None) == 'R'],
                'vs_LHB': [play_matchup.get('batSide', {}).get('code', None) == 'L'],
                'vs_SHB': [play_matchup.get('batSide', {}).get('code', None) == 'S'],
                'vs_RHP': [play_matchup.get('pitchHand', {}).get('code', None) == 'R'],
                'vs_LHP': [play_matchup.get('pitchHand', {}).get('code', None) == 'L'],
                'play_events': [play_events]
            })
            game_data_list.append(play_data)

    return game_data_list

def fetch_and_process_games(season):
    games_list = top_players_games[top_players_games['season'] == season]['gamePk'].tolist()
    batter_list = batting_season_team[batting_season_team['season'] == season]['player_id'].tolist()
    pitcher_list = pitching_season_team[pitching_season_team['season'] == season]['player_id'].tolist()

    all_game_data = []

    with ThreadPoolExecutor() as executor:
        futures = []
        for game in games_list:
            futures.append(executor.submit(process_game, game, season, batter_list, pitcher_list))

        for future in tqdm(as_completed(futures), total=len(futures), desc=f"Processing Season {season}"):
            game_data = future.result()
            all_game_data.extend(game_data)

    return all_game_data

# There is only full play data from 1950 up to recent years
for season in games_seasons:
    season_data = fetch_and_process_games(season)
    plays_results.extend(season_data)

Processing Season 2024: 100%|██████████| 2382/2382 [01:26<00:00, 27.63it/s]


In [69]:
top_players_plays = pd.concat(plays_results, ignore_index=True)

In [10]:
top_players_plays.to_csv('mlb_top_players_plays.csv', index=False)